In [1]:
import os
import sys
sys.path.append('../../')
from recs.backtest import get_backtest_answer
from recs.utils.path import *
import __init__
from recs.utils.common import predict_ranking, readjson2dict, recall_evaluate
import recs
from tqdm import tqdm
import pandas as pd
tqdm.pandas()

In [2]:
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
real = get_backtest_answer(tran_data, start=20200000, end=20210000)

100%|██████████| 28496/28496 [01:14<00:00, 380.94it/s]


In [3]:
target_date = 202000
trans_buy = tran_data[tran_data.buy_date < (target_date)*100]
crm = pd.read_pickle(DATA_PATH / "crm_diff.pkl")
print(crm.columns)
target_users = crm[crm.yyyymm == 201912].id_number.unique().tolist()
best_sells = trans_buy[
    (trans_buy.buy_date > 20171200 )
].fund_id.value_counts().rename_axis('fund_id').reset_index(name='count')
trans_buy

Index(['yyyymm', 'id_number', 'local_foreign_total', 'local_total',
       'local_demand_deposit', 'local_fixed_deposit', 'foreign_total',
       'foreign_fixed_deposit', 'foreign_demand_deposit', 'invest_type', 'age',
       'monthly_trade_vol', 'stock_inventory_val', 'KPI',
       'local_foreign_total_diff', 'local_total_diff',
       'local_demand_deposit_diff', 'local_fixed_deposit_diff',
       'foreign_total_diff', 'foreign_fixed_deposit_diff',
       'foreign_demand_deposit_diff', 'monthly_trade_vol_diff',
       'stock_inventory_val_diff', 'age_group'],
      dtype='object')


,id_number,fund_id,buy_date,yyyymm
1,13627,930,20170327,201702
5,13651,748,20190709,201906
8,13651,2053,20170926,201708
11,13653,2518,20170616,201705
12,13654,2091,20160322,201602
...,...,...,...,...
395706,67934,3041,20180116,201712
395707,67951,2108,20191114,201910
395708,67969,482,20180509,201804
395709,67971,2058,20180515,201804


In [4]:
### for VAE
# vaecf = recs.models.VAECF(
#     k=10,
#     autoencoder_structure=[20],
#     act_fn="tanh",
#     likelihood="mult",
#     n_epochs=100,
#     batch_size=100,
#     learning_rate=0.001,
#     beta=1.0,
#     seed=123,
#     use_gpu=True,
#     verbose=True,
# )
# trans_buy['t_weight'] = trans_buy.buy_date.progress_apply(lambda x: pow((x - trans_buy.buy_date.min() + 1) / (trans_buy.buy_date.max() - trans_buy.buy_date.min()), 2))
# rate_df = trans_buy.groupby(["id_number", "fund_id"]).agg({'t_weight':'sum'}).reset_index().rename(columns={"t_weight": "rating"})
# uir_data = rate_df[['id_number', 'fund_id', 'rating']].values
# train_set = recs.datasets.Dataset.from_uir(uir_data, seed=41)
# df = pd.DataFrame(columns=['userId', 'itemId', 'rating'], data=uir_data)

# vaecf.fit(train_set)
# pred = predict_ranking(vaecf, df, 'userId', 'itemId', 'rating', False)
# pred.sort_values(by='rating', ascending=False, inplace=True)
# pred



In [5]:
### for VAE
# rec_dict = {}
# for u in tqdm(target_users):
#     temp = pred[pred.userId == u]
#     if len(temp) == 0:
# #         ignore_items = trans_buy[trans_buy.id_number == u].fund_id.unique().tolist()        
# #         rec_dict[u] = best_sells[~best_sells['fund_id'].isin(ignore_items)]['fund_id'].tolist()[:10]
#         rec_dict[u] = best_sells['fund_id'].tolist()[:10]
#     else:
#         rec_dict[u] = temp.itemId.unique().tolist()[:10]
# for i in tqdm(rec_dict):
#     rec_dict[i] = [int(x) for x in rec_dict[i]]

In [19]:
# import json
# from recs.utils.common import convert_result_json_2_csv
# r_name = "rb_2020_vaecf"
# json_name = r_name+".json"
# json_save_path = RESULT_PATH / json_name 
# # with open(json_save_path, 'w') as jf:
# #     json.dump(rec_dict, jf)

# csv_name = f"{r_name}.csv"
# save_path = RESULT_PATH / csv_name
# # convert_result_json_2_csv(str(json_save_path), str(save_path))

# vaecf_data = pd.read_csv(save_path)

In [26]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.cluster import Birch
import itertools
import numpy as np
import umap
def normalized(x):
    return (x - x.mean()) / x.std()

n_cluster = 100

user_data = crm[crm.yyyymm == 201912]
udata = user_data[['local_foreign_total', 'local_total',
       'local_demand_deposit', 'local_fixed_deposit', 'foreign_total',
       'foreign_fixed_deposit', 'foreign_demand_deposit', 'invest_type', 'age',
       'monthly_trade_vol', 'stock_inventory_val', 'KPI','local_foreign_total_diff', 'local_total_diff',
       'local_demand_deposit_diff', 'local_fixed_deposit_diff',
       'foreign_total_diff', 'foreign_fixed_deposit_diff',
       'foreign_demand_deposit_diff', 'monthly_trade_vol_diff',
       'stock_inventory_val_diff', 'age_group']] 
for i in ['local_foreign_total_diff', 'local_total_diff',
       'local_demand_deposit_diff', 'local_fixed_deposit_diff',
       'foreign_total_diff', 'foreign_fixed_deposit_diff',
       'foreign_demand_deposit_diff', 'monthly_trade_vol_diff',
       'stock_inventory_val_diff']:
    udata[i] = normalized(udata[i])
udata = udata.replace(-np.inf, -1)
udata.fillna(0, inplace=True)
# x_embedded = TSNE(n_components=3, init='random', random_state=47, n_jobs=30).fit_transform(udata.values)
# x_embedded = np.array(x_embedded)


x_embedded = umap.UMAP(n_neighbors=20, n_components=10, metric='cosine').fit_transform(udata.values)
user_data.columns

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Index(['yyyymm', 'id_number', 'local_foreign_total', 'local_total',
       'local_demand_deposit', 'local_fixed_deposit', 'foreign_total',
       'foreign_fixed_deposit', 'foreign_demand_deposit', 'invest_type', 'age',
       'monthly_trade_vol', 'stock_inventory_val', 'KPI',
       'local_foreign_total_diff', 'local_total_diff',
       'local_demand_deposit_diff', 'local_fixed_deposit_diff',
       'foreign_total_diff', 'foreign_fixed_deposit_diff',
       'foreign_demand_deposit_diff', 'monthly_trade_vol_diff',
       'stock_inventory_val_diff', 'age_group'],
      dtype='object')

In [27]:
# x_embedded = udata.values
brc = Birch(n_clusters=n_cluster)
brc.fit(x_embedded)
brc_x = np.array(brc.predict(x_embedded))
user_data['cluster'] = brc_x

tran_data = trans_buy.copy()


cluster_rec = {}
for i in tqdm(range(n_cluster)):
    userlen = len(user_data[user_data['cluster'] == i])
    tranlen = len(tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['id_number'])])
    temp = tran_data[tran_data['id_number'].isin( user_data[user_data['cluster'] == i]['id_number'])]['fund_id'].value_counts().index.tolist()[:10]
    cluster_rec[i] = temp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
100%|██████████| 100/100 [00:00<00:00, 101.06it/s]


In [10]:
# cluster_rec_vae = {}
# for i in tqdm(range(n_cluster)):
#     count_dict = {}

#     temp = vaecf_data[vaecf_data['uid'].isin(user_data[user_data['cluster'] == i]['id_number'])]

#     for j in range(10):
#         temp_count_dict = temp[f"fundid{j}"].value_counts().to_dict()
#         for k in temp_count_dict.keys():
#             if count_dict.get(k):
#                 count_dict[k] += int(temp_count_dict[k])
#             else:
#                 count_dict[k] = int(temp_count_dict[k])


#     result = dict(sorted(count_dict.items(), key=lambda item: item[1], reverse=True))    
#     cluster_rec_vae[i] = list(result.keys())[0:10]

In [28]:
cluster_rec_dict = {}
cvae_rec_dict = {}
for u in tqdm(target_users):
    temp = user_data[user_data['id_number']==u]
    t_cluster = temp.cluster.tolist()[0]    
    cluster_rec_dict[u] = cluster_rec[t_cluster]
#     cvae_rec_dict[u] = cluster_rec_vae[t_cluster]
    

100%|██████████| 69839/69839 [00:36<00:00, 1912.16it/s]


In [18]:
tran_data = pd.read_pickle(DATA_PATH / "trans_buy.pkl")
user2020 = list(tran_data[tran_data.buy_date > 20200000].id_number.unique())
user2017 =list(tran_data[tran_data.buy_date < 20200000].id_number.unique())
new_user = list(set(user2020) - set(user2017))

new_real = {}
for i in tqdm(real):
    if i in new_user:
        new_real[i] = real[i]

100%|██████████| 28496/28496 [00:42<00:00, 673.69it/s]


In [24]:
# umap 2

temp_dict = {}
for i in cluster_rec_dict:
    temp_dict[int(i)] = best_sells['fund_id'][:10]
    
print(recall_evaluate(cluster_rec_dict, new_real))
print(recall_evaluate(cvae_rec_dict, new_real))
print(recall_evaluate(temp_dict, new_real))

100%|██████████| 11056/11056 [00:00<00:00, 269425.82it/s]


{'recall@3': 0.009295343916816422, 'recall@5': 0.011968245364138999, 'recall@10': 0.018394368937060684, 'recall@all': 0.018394368937060684}


100%|██████████| 11056/11056 [00:00<00:00, 305156.72it/s]


{'recall@3': 0.0, 'recall@5': 0.0, 'recall@10': 0.0, 'recall@all': 0.0}


100%|██████████| 11056/11056 [00:00<00:00, 45053.73it/s]

{'recall@3': 0.00816423360969309, 'recall@5': 0.010793052772520937, 'recall@10': 0.013169624776164578, 'recall@all': 0.013169624776164578}


In [20]:
# tsne 3

temp_dict = {}
for i in cluster_rec_dict:
    temp_dict[int(i)] = best_sells['fund_id'][:10]
    
print(recall_evaluate(cluster_rec_dict, new_real))
print(recall_evaluate(cvae_rec_dict, new_real))
print(recall_evaluate(temp_dict, new_real))

100%|██████████| 11056/11056 [00:00<00:00, 277420.51it/s]


{'recall@3': 0.008932903354484397, 'recall@5': 0.011904391937857932, 'recall@10': 0.018104787508278826, 'recall@all': 0.018104787508278826}


100%|██████████| 11056/11056 [00:00<00:00, 312586.62it/s]


{'recall@3': 0.0, 'recall@5': 0.0, 'recall@10': 0.0, 'recall@all': 0.0}


100%|██████████| 11056/11056 [00:00<00:00, 45586.45it/s]

{'recall@3': 0.00816423360969309, 'recall@5': 0.010793052772520937, 'recall@10': 0.013169624776164578, 'recall@all': 0.013169624776164578}


In [25]:
# umap 5

temp_dict = {}
for i in cluster_rec_dict:
    temp_dict[int(i)] = best_sells['fund_id'][:10]
    
print(recall_evaluate(cluster_rec_dict, new_real))
print(recall_evaluate(cvae_rec_dict, new_real))
print(recall_evaluate(temp_dict, new_real))

100%|██████████| 11056/11056 [00:00<00:00, 269981.11it/s]


{'recall@3': 0.009762264739200343, 'recall@5': 0.01177751362040436, 'recall@10': 0.01905557710098231, 'recall@all': 0.01905557710098231}


100%|██████████| 11056/11056 [00:00<00:00, 313289.86it/s]


{'recall@3': 0.0, 'recall@5': 0.0, 'recall@10': 0.0, 'recall@all': 0.0}


100%|██████████| 11056/11056 [00:00<00:00, 44072.02it/s]

{'recall@3': 0.00816423360969309, 'recall@5': 0.010793052772520937, 'recall@10': 0.013169624776164578, 'recall@all': 0.013169624776164578}


In [29]:
# umap 10

temp_dict = {}
for i in cluster_rec_dict:
    temp_dict[int(i)] = best_sells['fund_id'][:10]
    
print(recall_evaluate(cluster_rec_dict, new_real))
print(recall_evaluate(cvae_rec_dict, new_real))
print(recall_evaluate(temp_dict, new_real))

100%|██████████| 11056/11056 [00:00<00:00, 282550.73it/s]


{'recall@3': 0.009250622096588088, 'recall@5': 0.012124944670621227, 'recall@10': 0.01837847190596829, 'recall@all': 0.01837847190596829}


100%|██████████| 11056/11056 [00:00<00:00, 307869.48it/s]


{'recall@3': 0.0, 'recall@5': 0.0, 'recall@10': 0.0, 'recall@all': 0.0}


100%|██████████| 11056/11056 [00:00<00:00, 41511.03it/s]

{'recall@3': 0.00816423360969309, 'recall@5': 0.010793052772520937, 'recall@10': 0.013169624776164578, 'recall@all': 0.013169624776164578}


In [39]:
list(rec_dict.keys())[0]

'38831'

In [40]:
rec_dict['38831']

[2936.0,
 3215.0,
 2940.0,
 2717.0,
 3197.0,
 3041.0,
 2416.0,
 2518.0,
 1108.0,
 3072.0]